In [61]:
import pyexr
from matplotlib import pyplot as plt
from src.model.shape_model import ShapeNetwork
from src.model.illumination_model import IlluminationNetwork
from src.model.svbrdf_model import SVBRDF_Network
from src.model.joint_model import JointNetwork
from src.utils.visualize_tools import save_img
from pathlib import Path
from PIL import Image
import torch
import numpy as np
import os

In [62]:
def load_rgb(path : Path):
    """Read an image using Pillow

    Parameters
    ----------
    path : Path
        File path.
        
    Returns
    -------
    img : RGB or RGBA image in float32 format.
    """
    image_ = Image.open(str(path))
    img = np.array(image_.getdata(), dtype=np.float32)
    img = img.reshape((256, 256, -1))
    img = img[:, :, 0:3]
    img = img / 255.0
    return img

In [63]:
def load_mono(path : Path, ch=0):
    """Read an image using Pillow, return n-th channel

    Parameters
    ----------
    path : Path
        File path.
    ch   : int
        optional channel id
        
    Returns
    -------
    channel : channel in float32 format
    """
    channel = load_rgb(path)
    channel = channel[..., ch]
    return channel

In [64]:
#model = ShapeNetwork()
model = ShapeNetwork.load_from_checkpoint(checkpoint_path="lightning_logs\\version_139\\checkpoints\\epoch=3-step=6187.ckpt")

In [65]:
#test_sample["cam1"] = torch.Tensor(test_sample["cam1"][None, ...])
#test_sample["cam2"] = torch.Tensor(test_sample["cam2"][None, ...])
#test_sample["mask"] = torch.Tensor(test_sample["mask"][None, ...])
cam1_path = Path("Test_Results/0/cam1.png")
cam2_path = Path("Test_Results/0/cam2.png")
mask_path = Path("Test_Results/0/mask.png")
cam1 = torch.Tensor(load_rgb(cam1_path)[None, ...]).permute((0, 3, 1, 2))
cam2 = torch.Tensor(load_rgb(cam2_path)[None, ...]).permute((0, 3, 1, 2))
mask = load_mono(mask_path)[None, None, ...]
mask = torch.Tensor(np.where(np.less(mask, 0.9), np.zeros_like(mask), np.ones_like(mask)))
normal, depth = model.forward((cam1, cam2, mask))

In [66]:
shape_path = "Test_Results/Test/shape/"

save_img(depth, shape_path, "depth")
save_img(normal, shape_path, "normal")
save_img(mask, shape_path, "mask")
save_img(cam1, shape_path, "cam1")
save_img(cam2, shape_path, "cam2")


#normal = normal.permute(0, 2, 3, 1).squeeze(0)
#depth = depth.squeeze(0).squeeze(0)
#normal = normal.detach().cpu().numpy()
#depth = depth.detach().cpu().numpy()

# create a new folder Test_Results
# and save the depth and normal maps
#if not os.path.exists("Test_Results/Test"):
#    os.makedirs("Test_Results/Test")

# save the depth map using matplotlib
#plt.imsave("Test_Results/Test/depth.png", depth, cmap="gray", vmin=0, vmax=1)

# save the normal map as rgb using matplotlib
#plt.imsave("Test_Results/Test/normal.png", normal)

#cam1 = cam1.squeeze(0).permute((1, 2, 0)).detach().cpu().numpy()
#cam2 = cam2.squeeze(0).permute((1, 2, 0)).detach().cpu().numpy()
#mask = mask.detach().cpu().numpy().reshape(256, 256, 1)
#plt.imsave("Test_Results/Test/cam1.png", cam1)
#plt.imsave("Test_Results/Test/cam2.png", cam2)
#plt.imsave("Test_Results/2/mask.png", mask, cmap="gray")

In [67]:
illumination_model = IlluminationNetwork.load_from_checkpoint(checkpoint_path="lightning_logs\\version_155\\checkpoints\\epoch=7-step=12375.ckpt")

In [68]:
path_illum = "Test_Results/Test/illumination/"

sgs = illumination_model.forward((cam1, cam2, mask, normal, depth))

# no saving here

In [69]:
svbrdf_model = SVBRDF_Network.load_from_checkpoint(checkpoint_path="lightning_logs\\version_158\\checkpoints\\epoch=2-step=4640.ckpt")
path_svbrdf = "Test_Results/Test/svbrdf/"

diffuse, specular, roughness = svbrdf_model.forward((cam1, cam2, mask, normal, depth))

save_img(diffuse, path_svbrdf, "diffuse")
save_img(specular, path_svbrdf, "specular")
save_img(roughness, path_svbrdf, "roughness")

TypeError: apply_mask() missing 1 required positional argument: 'name'